In [1]:
import os
import pandas as pd
from pandas_gbq import to_gbq
from google.cloud import bigquery


import re
##import datetime 


#import numpy as np
#import pandas_gbq
#import janitor

#from google.oauth2 import service_account

### Configure Goggle Big Query

In [2]:
gbq_project_ID = "wedge-project-np"
gbq_dataset_ID = "wedge"

client = bigquery.Client(project = gbq_project_ID)
credentials = client._credentials

### Set Data Path

In [3]:
data_directory = "Data/processed_files/"

### Setup Schema

In [5]:
# Define the Big Query schema
schema = [
    bigquery.SchemaField("datetime", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("register_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("Scale", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("ItemQtty", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volDiscType", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("VolSpecial", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memType", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("staff", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("charflag", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("varflag", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("batchHeaderID", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("display", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("match_ID", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_ID", "FLOAT", mode="NULLABLE")
]

# Set the schema in the job config
job_config = bigquery.LoadJobConfig(schema=schema)

### Configure Big Query

In [5]:
#job_config = bigquery.LoadJobConfig()
#job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
#job_config.schema_update_options = [bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION]
#skip_leading_rows=1,  # Skip header row

#job_config.skip_leading_rows = 1


job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip header row
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE,
    schema_update_options = [bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION]  
)

In [6]:
clean_files = os.listdir(data_directory)

In [11]:
for file_name in clean_files:
    df= pd.read_csv(data_directory + file_name)
    table_name = file_name.replace("_processed.csv", "")

    full_table_name = f"{gbq_dataset_ID}.{table_name}"
    to_gbq(df, full_table_name, project_id=gbq_project_ID, if_exists="replace", credentials=credentials)

100%|██████████| 1/1 [00:00<?, ?it/s]
C:\Users\npleg\AppData\Local\Temp\ipykernel_15628\3771781157.py:2: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(data_directory + file_name)
Error converting Pandas column with name: "organic" and datatype: "object" to an appropriate pyarrow datatype: Array, ListArray, or StructArray


ArrowTypeError: Error converting Pandas column with name: "organic" and datatype: "object" to an appropriate pyarrow datatype: Array, ListArray, or StructArray

In [9]:
full_table_name = f"{gbq_dataset_ID}.{table_name}"
to_gbq(df, full_table_name, project_id=gbq_project_ID, if_exists="replace", credentials=credentials)

100%|██████████| 1/1 [00:00<?, ?it/s]


### Loop over to upload all Processed Files

In [ ]:
for file_name in os.listdir(data_directory):
    if file_name.endswith("_small.csv_processed.csv"):
        # Construct full path and table name
        full_path = os.path.join(data_directory, file_name)
        table_name = file_name.replace("_small.csv_processed.csv", "")  # Remove suffix
        full_table_name = f"{gbq_dataset_ID}.{table_name}"  # Full table name

        # Check if the file exists
        if os.path.exists(full_path):
            print(f"Uploading {file_name} to BigQuery...")

            # Open the CSV and upload to BigQuery
            with open(full_path, 'rb') as source_file:
                job = client.load_table_from_file(
                    source_file,
                    full_table_name,
                    job_config=job_config,
                )
                job.result()  # Wait for the job to complete
                print(f"{file_name} uploaded successfully to {full_table_name}.")
            break
        else:
            print(f"File not found: {file_name}")

In [ ]:
full_path= data_directory + current_zf
if os.path.exists(full_path):
    print("File found")

    with open(data_directory + current_zf, 'rb') as source_file:
        job = client.load_table_from_file(
            source_file,
            f"{gbq_dataset_id}.{current_zf}",
            job_config=job_config,
        )
else:
    print("File not found")

In [ ]:
full_path= data_directory + current_zf
if os.path.exists(full_path):
    print("File found")

    with open(data_directory + current_zf, 'rb') as source_file:
        job = client.load_table_from_file(
            source_file,
            f"{gbq_dataset_id}.{current_zf}",
            job_config=job_config,
        )
else:
    print("File not found")

In [ ]:

for file in data_directory:
    tab, _ = file.replace("_clean.csv", "")
    table_full_name = ".".join([gbq_project_id, gbq_dataset_id, tab])

    if not table_exists(client, table_full_name):
        table_ref = client.create_table(table=table_full_name)
    else:
        table_ref = client.get_table(table_full_name)

    table = client.get_table(table_ref)
    print("Table {} contains {} columns".format(table_ref.table_id, len(table.schema)))

In [ ]:
for file_name in clean_files:
    df = pd.read_csv(data_path + file_name)
    
    # Replace NaN and empty strings with 'Unknown' in 'memType'
    df['memType'] = df['memType'].fillna('Unknown').replace('', 'Unknown').astype(str)
    
    # Convert 'datetime' column to datetime type
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    
    # Fill NaN values in numeric columns only if they exist
    numeric_columns = ['taxexempt', 'itemstatus', 'tenderstatus', 'batchHeaderID']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].fillna(0)
 
    # Proceed with uploading to BigQuery
    table_name = file_name.replace("_clean.csv", "")
    full_table_name = f'{dataset_id}.{table_name}'
    
    # Upload to GBQ
    to_gbq(df, full_table_name, project_id=project_id, credentials=credentials, if_exists='replace')
 